# Portfolio Analyzer

In [1]:
# Filter warnings + watermark
from watermark import watermark
from warnings import filterwarnings
filterwarnings("ignore")
print(watermark())
print(watermark(iversions=True, globals_=globals()))

Last updated: 2022-11-16T19:20:51.681660+00:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.31.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.0-1094-azure
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit




In [2]:
pip install polygon-api-client

Note: you may need to restart the kernel to use updated packages.


In [3]:
#from polygon import RESTClient

In [4]:
#import the necessary modules
import pandas as pd
import numpy as np
#questionary
from pathlib import Path
import fire
import questionary
#API
import os
import json
import requests
from dotenv import load_dotenv
from MCForecastTools import MCSimulation
%matplotlib inline


In [5]:
from IPython.display import display_html 

In [6]:
POLYGON_API_KEY ='enZp2AUpH4pGXJJuQ1CbjdVXJIsBFBEl'

In [7]:
# Load .env enviroment variables into the notebook
#load_dotenv()

In [8]:
# Get the API key from the environment variable and store as Python variable
#polygon_api_key = os.getenv("POLYGON_API_KEY")

# Using the type funcion, confirm that the Nasdaq API key is available for use in the notebook
#type(polygon_api_key)

In [9]:
ticker_list = ["AMZN", "RTH", "AMT", "IYR", "XOM", "XLE", "SPY"]

In [10]:
import requests
import datetime
from time import sleep
from tqdm import tqdm
def get_prices(start_date,end_date,universe):
    data = pd.DataFrame()
    # end_at = datetime.datetime.now() 
    # begin_from = end_at + datetime.timedelta(days=-5000)
    for t in tqdm(universe):
        print(t)
        print(len(data))
        # for start in pd.date_range(start = begin_from, end = end_at,normalize=True,freq = '88D'):
        #     end = start + datetime.timedelta(days = 88)
        #     end = str(int((end - datetime.datetime(1970,1,1)).total_seconds()))
        #     start = str(int((start - datetime.datetime(1970,1,1)).total_seconds()))
        try:
            r = requests.get(f'https://api.polygon.io/v2/aggs/ticker/{t}/range/1/day/{start_date}/{end_date}?apiKey={POLYGON_API_KEY}')
        except json.JSONDecodeError:
            pass
        try:
            # print(r.json())
            temp_data = pd.DataFrame(r.json()['results'])
            temp_data['ticker'] = t
            data = data.append(temp_data, ignore_index=True)
            print(t+':'+str(pd.to_datetime(start_date, unit='s'))+':'+str(pd.to_datetime(end_date, unit='s')))
        except KeyError:
            print(f'{t} was not found')
            pass
        except ValueError:
            pass
        except ConnectionError as error:
            print(error)
            sleep(200)
            continue
        except TimeoutError as error:
            print(error)
            sleep(200)
            continue
        sleep(0.25)
#     data.to_csv(file_name,index = False)
    data.t = pd.to_datetime(data.t, unit = 'ms')
    columns_name = {'t': 'time', 'o': 'Open', 'c': 'Close', 'h': 'High', 'l': 'Low', 'v': 'Volume'} 
    data= data.rename(columns = columns_name)
    data = data[['time','ticker','Open', 'Close','High', 'Low', 'Volume']]
    data = data.set_index(['time', 'ticker'])
    data = data.pct_change()
    data["daily_returns"] = data['Close'].groupby('ticker').pct_change()
    #data.to_csv(file_name, index = False)
    return data

In [11]:
#f"https://api.polygon.io/v2/aggs/ticker/{t}/range/1/day/{start_date}/{end_date}?apiKey=rHtIrGbGhJY6umnXSqp6hYgTM3XnHI2_"

In [12]:
# Create `start_date` and `end_date` variables for the period before the pandemic
start_date = "2017-03-01"
end_date = "2020-03-01"
all_data = get_prices(start_date=start_date, end_date=end_date, universe=ticker_list)
#example
#big_mac_usa = "https://data.nasdaq.com/api/v3/datasets/ECONOMIST/BIGMAC_USA?start_date=" + start_date + "&end_date=" + end_date + "&api_key=" + nasdaq_api_key

  0%|                                                                                                                                                                              | 0/7 [00:00<?, ?it/s]

AMZN
0


 14%|███████████████████████▋                                                                                                                                              | 1/7 [00:00<00:03,  1.65it/s]

RTH
572


 29%|███████████████████████████████████████████████▍                                                                                                                      | 2/7 [00:01<00:03,  1.26it/s]

...

 71%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                               | 5/7 [00:03<00:01,  1.46it/s]

XLE
2860


 86%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                       | 6/7 [00:04<00:00,  1.53it/s]

SPY
3432


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.50it/s]


In [13]:
all_data.index

MultiIndex([('2017-11-17 05:00:00', 'AMZN'),
            ('2017-11-20 05:00:00', 'AMZN'),
            ('2017-11-21 05:00:00', 'AMZN'),
            ('2017-11-22 05:00:00', 'AMZN'),
            ('2017-11-24 05:00:00', 'AMZN'),
            ('2017-11-27 05:00:00', 'AMZN'),
            ('2017-11-28 05:00:00', 'AMZN'),
            ('2017-11-29 05:00:00', 'AMZN'),
            ('2017-11-30 05:00:00', 'AMZN'),
            ('2017-12-01 05:00:00', 'AMZN'),
            ...
            ('2020-02-14 05:00:00',  'SPY'),
            ('2020-02-18 05:00:00',  'SPY'),
            ('2020-02-19 05:00:00',  'SPY'),
            ('2020-02-20 05:00:00',  'SPY'),
            ('2020-02-21 05:00:00',  'SPY'),
            ('2020-02-24 05:00:00',  'SPY'),
            ('2020-02-25 05:00:00',  'SPY'),
            ('2020-02-26 05:00:00',  'SPY'),
            ('2020-02-27 05:00:00',  'SPY'),
            ('2020-02-28 05:00:00',  'SPY')],
           names=['time', 'ticker'], length=4004)

In [14]:
idx = pd.IndexSlice

In [15]:
AMZN_daily_returns_df = all_data.loc[idx[:,'AMZN'],['daily_returns']]
AMZN_daily_returns_df

,,daily_returns
time,ticker,
2017-11-17 05:00:00,AMZN,NaN
2017-11-20 05:00:00,AMZN,NaN
2017-11-21 05:00:00,AMZN,-4.703579
2017-11-22 05:00:00,AMZN,0.250166
2017-11-24 05:00:00,AMZN,0.764232
...,...,...
2020-02-24 05:00:00,AMZN,0.558597
2020-02-25 05:00:00,AMZN,-0.560144
2020-02-26 05:00:00,AMZN,-1.190887


In [16]:
RTH_daily_returns_df = all_data.loc[idx[:,'RTH'],['daily_returns']]
RTH_daily_returns_df

,,daily_returns
time,ticker,
2017-11-17 05:00:00,RTH,NaN
2017-11-20 05:00:00,RTH,-0.993868
2017-11-21 05:00:00,RTH,-6.403172
2017-11-22 05:00:00,RTH,0.402957
2017-11-24 05:00:00,RTH,-1.000000
...,...,...
2020-02-24 05:00:00,RTH,2.086610
2020-02-25 05:00:00,RTH,-0.108921
2020-02-26 05:00:00,RTH,-0.917572


In [17]:
AMT_daily_returns_df = all_data.loc[idx[:,'AMT'],['daily_returns']]
AMT_daily_returns_df

,,daily_returns
time,ticker,
2017-11-17 05:00:00,AMT,NaN
2017-11-20 05:00:00,AMT,-0.991905
2017-11-21 05:00:00,AMT,0.054612
2017-11-22 05:00:00,AMT,-4.450728
2017-11-24 05:00:00,AMT,-2.568382
...,...,...
2020-02-24 05:00:00,AMT,2.592598
2020-02-25 05:00:00,AMT,-1.546307
2020-02-26 05:00:00,AMT,-4.354834


In [18]:
IYR_daily_returns_df = all_data.loc[idx[:,'IYR'],['daily_returns']]
IYR_daily_returns_df

,,daily_returns
time,ticker,
2017-11-17 05:00:00,IYR,NaN
2017-11-20 05:00:00,IYR,-0.996755
2017-11-21 05:00:00,IYR,-4.477786
2017-11-22 05:00:00,IYR,-1.387045
2017-11-24 05:00:00,IYR,-1.741195
...,...,...
2020-02-24 05:00:00,IYR,-8.039260
2020-02-25 05:00:00,IYR,1.004415
2020-02-26 05:00:00,IYR,-0.608371


In [19]:
XOM_daily_returns_df = all_data.loc[idx[:,'XOM'],['daily_returns']]
XOM_daily_returns_df

,,daily_returns
time,ticker,
2017-11-17 05:00:00,XOM,NaN
2017-11-20 05:00:00,XOM,-1.047159
2017-11-21 05:00:00,XOM,0.028285
2017-11-22 05:00:00,XOM,-0.284094
2017-11-24 05:00:00,XOM,0.387359
...,...,...
2020-02-24 05:00:00,XOM,2.841366
2020-02-25 05:00:00,XOM,-0.181892
2020-02-26 05:00:00,XOM,-0.427118


In [20]:
XLE_daily_returns_df = all_data.loc[idx[:,'XLE'],['daily_returns']]
XLE_daily_returns_df

,,daily_returns
time,ticker,
2017-11-17 05:00:00,XLE,NaN
2017-11-20 05:00:00,XLE,-1.007633
2017-11-21 05:00:00,XLE,-1.939730
2017-11-22 05:00:00,XLE,0.795995
2017-11-24 05:00:00,XLE,-0.299103
...,...,...
2020-02-24 05:00:00,XLE,2.892330
2020-02-25 05:00:00,XLE,-0.043848
2020-02-26 05:00:00,XLE,-0.316305


In [21]:
SPY_daily_returns_df = all_data.loc[idx[:,'SPY'],['daily_returns']]
SPY_daily_returns_df

,,daily_returns
time,ticker,
2017-11-17 05:00:00,SPY,NaN
2017-11-20 05:00:00,SPY,-0.999637
2017-11-21 05:00:00,SPY,2.834366
2017-11-22 05:00:00,SPY,-1.135210
2017-11-24 05:00:00,SPY,-3.611005
...,...,...
2020-02-24 05:00:00,SPY,2.220485
2020-02-25 05:00:00,SPY,-0.086334
2020-02-26 05:00:00,SPY,-0.878614


In [22]:
tech_daily_returns = pd.concat([AMZN_daily_returns_df, RTH_daily_returns_df], keys=["AMZN","RTH"])
tech_daily_returns

daily_returns
     time                ticker               
AMZN 2017-11-17 05:00:00 AMZN              NaN
     2017-11-20 05:00:00 AMZN              NaN
     2017-11-21 05:00:00 AMZN        -4.703579
     2017-11-22 05:00:00 AMZN         0.250166
     2017-11-24 05:00:00 AMZN         0.764232
...                                        ...
RTH  2020-02-24 05:00:00 RTH          2.086610
     2020-02-25 05:00:00 RTH         -0.108921
     2020-02-26 05:00:00 RTH         -0.917572
     2020-02-27 05:00:00 RTH         20.215298
     2020-02-28 05:00:00 RTH         -0.570715

[1144 rows x 1 columns]

In [23]:
RE_daily_returns = pd.concat([AMT_daily_returns_df, IYR_daily_returns_df], keys=["AMT","IYR"])
RE_daily_returns

daily_returns
    time                ticker               
AMT 2017-11-17 05:00:00 AMT               NaN
    2017-11-20 05:00:00 AMT         -0.991905
    2017-11-21 05:00:00 AMT          0.054612
    2017-11-22 05:00:00 AMT         -4.450728
    2017-11-24 05:00:00 AMT         -2.568382
...                                       ...
IYR 2020-02-24 05:00:00 IYR         -8.039260
    2020-02-25 05:00:00 IYR          1.004415
    2020-02-26 05:00:00 IYR         -0.608371
    2020-02-27 05:00:00 IYR          3.993005
    2020-02-28 05:00:00 IYR         -0.403613

[1144 rows x 1 columns]

In [24]:
energy_daily_returns = pd.concat([XOM_daily_returns_df, XLE_daily_returns_df], keys=["XOM","XLE"])
energy_daily_returns

daily_returns
    time                ticker               
XOM 2017-11-17 05:00:00 XOM               NaN
    2017-11-20 05:00:00 XOM         -1.047159
    2017-11-21 05:00:00 XOM          0.028285
    2017-11-22 05:00:00 XOM         -0.284094
    2017-11-24 05:00:00 XOM          0.387359
...                                       ...
XLE 2020-02-24 05:00:00 XLE          2.892330
    2020-02-25 05:00:00 XLE         -0.043848
    2020-02-26 05:00:00 XLE         -0.316305
    2020-02-27 05:00:00 XLE          0.771599
    2020-02-28 05:00:00 XLE         -0.983492

[1144 rows x 1 columns]

In [25]:
#Display all sectors of the stock/ETF dataframes
dfT_style = tech_daily_returns.style.set_table_attributes("style='display:inline; margin-right:20px;'").set_caption("Tech")
dfR_style = RE_daily_returns.style.set_table_attributes("style='display:inline'").set_caption("Real Estate")
dfE_style = energy_daily_returns.style.set_table_attributes("style='display:inline'").set_caption("Energy")


display_html(dfT_style._repr_html_() + dfR_style._repr_html_() + dfE_style._repr_html_(), raw=True)

Tech 
 
 
   
   
   
 daily_returns 
 
 
   
 time 
 ticker 
   
 
 
 
 
 AMZN 
 2017-11-17 05:00:00 
 AMZN 
 nan 
 
 
 2017-11-20 05:00:00 
 AMZN 
 nan 
 
 
 2017-11-21 05:00:00 
 AMZN 
 -4.703579 
 
 
 2017-11-22 05:00:00 
 AMZN 
 0.250166 
 
 
 2017-11-24 05:00:00 
 AMZN 
 0.764232 
 
 
 2017-11-27 05:00:00 
 AMZN 
 -0.678865 
 
 
 2017-11-28 05:00:00 
 AMZN 
 -1.224992 
 
 
 2017-11-29 05:00:00 
 AMZN 
 13.524844 
 
 
 2017-11-30 05:00:00 
 AMZN 
 -1.492142 
 
 
 2017-12-01 05:00:00 
 AMZN 
 -1.917995 
 
 
 2017-12-04 05:00:00 
 AMZN 
 0.996655 
 
 
 2017-12-05 05:00:00 
 AMZN 
 -1.275030 
 
 
 2017-12-06 05:00:00 
 AMZN 
 0.405255 
 
 
 2017-12-07 05:00:00 
 AMZN 
 -0.316289 
 
 
 2017-12-08 05:00:00 
 AMZN 
 -0.704863 
 
 
 2017-12-11 05:00:00 
 AMZN 
 2.125266 
 
 
 2017-12-12 05:00:00 
 AMZN 
 -1.551628 
 
 
 2017-12-13 05:00:00 
 AMZN 
 -0.751789 
 
 
 2017-12-14 05:00:00 
 AMZN 
 -11.671860 
 
 
 2017-12-15 05:00:00 
 AMZN 
 -0.522418 
 
 
 2017-12-18 05:00:00 
 AMZN 
 1.334560 
 
 
 2017-12-19 05:00:00 
 AMZN 
 -1.277033 
 
 
 2017-12-20 05:00:00 
 AMZN 
 2.058220 
 
 
 2017-12-21 05:00:00 
 AMZN 
 -0.704539 
 
 
 2017-12-22 05:00:00 
 AMZN 
 1.243210 
 
 
 2017-12-26 05:00:00 
 AMZN 
 -2.319690 
 
 
 2017-12-27 05:00:00 
 AMZN 
 -0.349912 
 
 
 2017-12-28 05:00:00 
 AMZN 
 -0.305066 
 
 
 2017-12-29 05:00:00 
 AMZN 
 -5.316708 
 
 
 2018-01-02 05:00:00 
 AMZN 
 -2.191693 
 
 
 2018-01-03 05:00:00 
 AMZN 
 -0.235396 
 
 
 2018-01-04 05:00:00 
 AMZN 
 -0.649637 
 
 
 2018-01-05 05:00:00 
 AMZN 
 2.610925 
 
 
 2018-01-08 05:00:00 
 AMZN 
 -0.107519 
 
 
 2018-01-09 05:00:00 
 AMZN 
 -0.675855 
 
 
 2018-01-10 05:00:00 
 AMZN 
 -0.721713 
 
 
 2018-01-11 05:00:00 
 AMZN 
 12.693838 
 
 
 2018-01-12 05:00:00 
 AMZN 
 0.253723 
 
 
 2018-01-16 05:00:00 
 AMZN 
 -1.011661 
 
 
 2018-01-17 05:00:00 
 AMZN 
 28.007556 
 
 
 2018-01-18 05:00:00 
 AMZN 
 -0.828317 
 
 
 2018-01-19 05:00:00 
 AMZN 
 -1.750974 
 
 
 2018-01-22 05:00:00 
 AMZN 
 24.950908 
 
 
 2018-01-23 05:00:00 
 AMZN 
 0.049840 
 
 
 2018-01-24 05:00:00 
 AMZN 
 -1.139084 
 
 
 2018-01-25 05:00:00 
 AMZN 
 -5.078675 
 
 
 2018-01-26 05:00:00 
 AMZN 
 0.161571 
 
 
 2018-01-29 05:00:00 
 AMZN 
 -0.362600 
 
 
 2018-01-30 05:00:00 
 AMZN 
 0.274341 
 
 
 2018-01-31 05:00:00 
 AMZN 
 -0.360133 
 
 
 2018-02-01 05:00:00 
 AMZN 
 -5.616793 
 
 
 2018-02-02 05:00:00 
 AMZN 
 -1.684842 
 
 
 2018-02-05 05:00:00 
 AMZN 
 -1.972062 
 
 
 2018-02-06 05:00:00 
 AMZN 
 -2.360668 
 
 
 2018-02-07 05:00:00 
 AMZN 
 -1.475125 
 
 
 2018-02-08 05:00:00 
 AMZN 
 1.590144 
 
 
 2018-02-09 05:00:00 
 AMZN 
 -0.827475 
 
 
 2018-02-12 05:00:00 
 AMZN 
 -5.312791 
 
 
 2018-02-13 05:00:00 
 AMZN 
 -0.413924 
 
 
 2018-02-14 05:00:00 
 AMZN 
 0.266247 
 
 
 2018-02-15 05:00:00 
 AMZN 
 -0.714277 
 
 
 2018-02-16 05:00:00 
 AMZN 
 -2.211414 
 
 
 2018-02-20 05:00:00 
 AMZN 
 -2.517779 
 
 
 2018-02-21 05:00:00 
 AMZN 
 -0.268824 
 
 
 2018-02-22 05:00:00 
 AMZN 
 -0.835537 
 
 
 2018-02-23 05:00:00 
 AMZN 
 5.047981 
 
 
 2018-02-26 05:00:00 
 AMZN 
 0.482638 
 
 
 2018-02-27 05:00:00 
 AMZN 
 -1.447663 
 
 
 2018-02-28 05:00:00 
 AMZN 
 -1.047452 
 
 
 2018-03-01 05:00:00 
 AMZN 
 -41.412970 
 
 
 2018-03-02 05:00:00 
 AMZN 
 -1.362448 
 
 
 2018-03-05 05:00:00 
 AMZN 
 2.419723 
 
 
 2018-03-06 05:00:00 
 AMZN 
 -0.408609 
 
 
 2018-03-07 05:00:00 
 AMZN 
 -0.480196 
 
 
 2018-03-08 05:00:00 
 AMZN 
 -0.072375 
 
 
 2018-03-09 05:00:00 
 AMZN 
 2.922815 
 
 
 2018-03-12 04:00:00 
 AMZN 
 -0.290930 
 
 
 2018-03-13 04:00:00 
 AMZN 
 -1.517202 
 
 
 2018-03-14 04:00:00 
 AMZN 
 -1.277975 
 
 
 2018-03-15 04:00:00 
 AMZN 
 -4.072558 
 
 
 2018-03-16 04:00:00 
 AMZN 
 0.232531 
 
 
 2018-03-19 04:00:00 
 AMZN 
 1.531118 
 
 
 2018-03-20 04:00:00 
 AMZN 
 -2.581306 
 
 
 2018-03-21 04:00:00 
 AMZN 
 -1.108902 
 
 
 2018-03-22 04:00:00 
 AMZN 
 6.967438 
 
 
 2018-03-23 04:00:00 
 AMZN 
 0.368171 
 
 
 2018-03-26 04:00:00 
 AMZN 
 -2.261956 
 
 
 2018-03-27 04:00:00 
 AMZN 
 -1.9374

### A) Analyzing Each Stock and ETF Individually


In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
from pytz import timezone
from IPython.display import clear_output
from time import sleep
import numpy as np
import pyfolio as pf

In [28]:
Pandemic_prices_df = pd.concat([AMZN_daily_returns_df, RTH_daily_returns_df, AMT_daily_returns_df, IYR_daily_returns_df, XOM_daily_returns_df, XLE_daily_returns_df, SPY_daily_returns_df],axis=1, keys=["AMZN","RTH","AMT","IYR","XOM","XLE","SPY"])
display(Pandemic_prices_df.std())
display(Pandemic_prices_df.mean())

AMZN  daily_returns   NaN
RTH   daily_returns   NaN
AMT   daily_returns   NaN
IYR   daily_returns   NaN
XOM   daily_returns   NaN
XLE   daily_returns   NaN
SPY   daily_returns   NaN
dtype: float64

AMZN  daily_returns   -inf
RTH   daily_returns    NaN
AMT   daily_returns    inf
IYR   daily_returns    NaN
XOM   daily_returns    inf
XLE   daily_returns    NaN
SPY   daily_returns    NaN
dtype: float64

In [29]:
#Summary statistics for AMZN stock using .std() and .mean()
display(AMZN_daily_returns_df.std())
display(AMZN_daily_returns_df.mean())

daily_returns   NaN
dtype: float64

daily_returns   -inf
dtype: float64

In [30]:
#Summary statistics for RTH stock using .std() and .mean()
RTH_daily_returns_df.std()
RTH_daily_returns_df.mean()

daily_returns   NaN
dtype: float64

In [31]:
#Summary statistics for AMT stock using .std() and .mean()
AMT_daily_returns_df.std()
AMT_daily_returns_df.mean()

daily_returns    inf
dtype: float64

In [32]:
#Summary statistics for IYR stock using .std() and .mean()
IYR_daily_returns_df.std()
IYR_daily_returns_df.mean()

daily_returns   NaN
dtype: float64

In [33]:
#Summary statistics for XOM stock using .std() and .mean()
XOM_daily_returns_df.std()
XOM_daily_returns_df.mean()

daily_returns    inf
dtype: float64

In [34]:
#Summary statistics for XLE stock using .std() and .mean()
XLE_daily_returns_df.std()
XLE_daily_returns_df.mean()

daily_returns   NaN
dtype: float64

In [35]:
#Summary statistics for AMZN using .describe()
AMZN_daily_returns_df.describe()

,daily_returns
count,570.000000
mean,-inf
std,NaN
min,-inf
25%,-1.817882
50%,-0.939097
75%,0.259872
max,1069.272420


In [36]:
#Summary statistics for RTH using .describe()
RTH_daily_returns_df.describe()

,daily_returns
count,571.000000
mean,NaN
std,NaN
min,-inf
25%,-1.916208
50%,-0.904449
75%,0.076291
max,inf


In [37]:
#Summary statistics for AMT using .describe()
AMT_daily_returns_df.describe()

,daily_returns
count,571.000000
mean,inf
std,NaN
min,-35.805961
25%,-1.906402
50%,-0.964839
75%,0.064229
max,inf


In [38]:
#Summary statistics for IYR using .describe()
IYR_daily_returns_df.describe()

,daily_returns
count,571.000000
mean,NaN
std,NaN
min,-inf
25%,-1.883685
50%,-0.970374
75%,0.064187
max,inf


In [39]:
#Summary statistics for XOM using .describe()
XOM_daily_returns_df.describe()

,daily_returns
count,571.000000
mean,inf
std,NaN
min,-99.986861
25%,-1.969918
50%,-0.981265
75%,0.077924
max,inf


In [40]:
#Summary statistics for XLE using .describe()
XLE_daily_returns_df.describe()

,daily_returns
count,570.000000
mean,NaN
std,NaN
min,-inf
25%,-2.007017
50%,-1.010595
75%,0.001510
max,inf


In [41]:
# Variance Of Amzn
AMZN_var=AMZN_daily_returns_df.var()
AMZN_var

daily_returns   NaN
dtype: float64

In [42]:
# Variance For RTH
RTH_var=RTH_daily_returns_df.var()
RTH_var

daily_returns   NaN
dtype: float64

In [43]:
# Variance Of IYR
IYR_var=IYR_daily_returns_df.var()
IYR_var

daily_returns   NaN
dtype: float64

In [44]:
# Variance Of AMT
AMT_var=AMT_daily_returns_df.var()
AMT_var

daily_returns   NaN
dtype: float64

In [45]:
# Variance Of XOM
XOM_var=XOM_daily_returns_df.var()
XOM_var

daily_returns   NaN
dtype: float64

In [46]:
# Variance Of XLE
XLE_var=XLE_daily_returns_df.var()
XLE_var

daily_returns   NaN
dtype: float64

In [47]:
# Covariance for Each Stock and ETF
AMZN_RTH_Cov=AMZN_daily_returns_df['daily_returns'].cov(RTH_daily_returns_df['daily_returns'])
AMZN_RTH_Cov

nan

In [48]:
# Covariance for Each Stock and ETF
AMT_IYR_Cov=AMT_daily_returns_df['daily_returns'].cov(IYR_daily_returns_df['daily_returns'])
AMT_IYR_Cov

nan

In [49]:
# Covariance for Each Stock and ETF
XOM_XLE_Cov=XOM_daily_returns_df['daily_returns'].cov(XLE_daily_returns_df['daily_returns'])
XOM_XLE_Cov

nan

In [50]:
# Annualized returns of RTH
RTH_Annual_Ret = RTH_daily_returns_df.mean()*252
RTH_Annual_Ret

daily_returns   NaN
dtype: float64

In [51]:
# Annualized returns of IYR
IYR_Annual_Ret = IYR_daily_returns_df.mean()*252
IYR_Annual_Ret

daily_returns   NaN
dtype: float64

In [52]:
# Annualized returns of XLE
XLE_Annual_Ret = XLE_daily_returns_df.mean()*252
XLE_Annual_Ret

daily_returns   NaN
dtype: float64

In [53]:
year_trading_days = 252

In [54]:
# Annualized std dev of RTH
annual_std_dev_RTH = RTH_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_RTH

daily_returns   NaN
dtype: float64

In [55]:
# Annualized std dev of IYR
annual_std_dev_IYR = IYR_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_IYR

daily_returns   NaN
dtype: float64

In [56]:
# Annualized std dev of XLE
annual_std_dev_XLE = XLE_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_XLE

daily_returns   NaN
dtype: float64

In [57]:
#Sharpe ratio for each stock and ETF 
Sharpe_RTH = RTH_Annual_Ret/annual_std_dev_RTH
Sharpe_RTH

daily_returns   NaN
dtype: float64

In [58]:
#Sharpe ratio for each stock and ETF 
Sharpe_IYR = IYR_Annual_Ret/annual_std_dev_IYR
Sharpe_IYR 

daily_returns   NaN
dtype: float64

In [59]:
#Sharpe ratio for each stock and ETF 
Sharpe_XLE = XLE_Annual_Ret/annual_std_dev_XLE
Sharpe_XLE

daily_returns   NaN
dtype: float64

In [60]:
# Calculate betas of AMZN
AMZN_beta = AMZN_RTH_Cov
# Display the beta of all stocks and ETFS
AMZN_beta

nan

In [61]:
# Calculate betas of RTH
RTH_beta = 
# Display the beta of all stocks and ETFS
RTH_beta

SyntaxError: invalid syntax (562119320.py, line 2)

In [ ]:
# Calculate betas of AMT
AMT_beta =
# Display the beta of all stocks and ETFS
AMT_beta

In [ ]:
# Calculate betas of IYR
IYR_beta = 
# Display the beta of all stocks and ETFS
IYR_beta

In [ ]:
# Calculate betas of XOM
XOM_beta = 
# Display the beta of all stocks and ETFS
XOM_beta

In [ ]:
# Calculate betas of XLE 
XLE_beta = 
# Display the beta of all stocks and ETFS
XLE_beta

In [ ]:
result = pf.create_returns_tear_sheet(AMZN_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("amzn_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(RTH_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("RTH_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(AMT_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("amt_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(IYR_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("iyr_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(XOM_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("xom_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(XLE_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("xle_prepan_tearsheet.png", format = "png")


In [ ]:
result = pf.create_returns_tear_sheet(SPY_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("spy_prepan_tearsheet.png", format = "png")

In [ ]:
# Calculate alphas of AMZN, RTH, AMT, IYR, XOM, XLE 

# Display the alphas of all stocks and ETFS


In [ ]:
# Calculate alphas of AMZN, RTH, AMT, IYR, XOM, XLE 

# Display the alphas of all stocks and ETFS


In [ ]:
# Calculate alphas of AMZN, RTH, AMT, IYR, XOM, XLE 

# Display the alphas of all stocks and ETFS


In [ ]:
# Calculate alphas of AMZN, RTH, AMT, IYR, XOM, XLE 

# Display the alphas of all stocks and ETFS


In [ ]:
# Calculate alphas of AMZN, RTH, AMT, IYR, XOM, XLE 

# Display the alphas of all stocks and ETFS


In [ ]:
# Calculate alphas of AMZN, RTH, AMT, IYR, XOM, XLE 



### A) Analyzing What Would Happen To A Portfolio in Each Sector For the Period Before the Pandemic

In [ ]:
#Questionary SQL
#what kind of investor are you (risk-averse, risk-neutral, risk loving)?
#assign portfolio to appropriate risk level given: risk-averse-all indexes, risk-neutral:mix, risk loving:all stocks

#What percentage of Tech would you like in your portfolio?
#What percentage of Real Estate would you like in your portfolio?
#What percentage of Energy would you like in your portfolio?

#What is your time horizon for investment? (short is more safe, long is riskier?)


In [ ]:
#Portfolio Returns for Tech: use (portfolio_return = weightAMZN * meanAMZN + weightRTH * meanRTH)


In [ ]:
#Portfolio Returns for Real Estate: use (portfolio_return = weightAMT * meanAMT + weightIYR * meanIYR)



In [ ]:
#Portfolio Returns for Energy: use (portfolio_return = weightXOM * meanXOM + weightXLE * meanXLE)



In [ ]:
# Using the Pandas var function, calculate the covariance of the S&P 500 using tech portfolio returns information
# The ETF SPY will represent the market


In [ ]:
# Using the Pandas var function, calculate the covariance of the S&P 500 using Real Estate portfolio returns information
# The ETF SPY will represent the market


In [ ]:
# Using the Pandas var function, calculate the covariance of the S&P 500 using Energy portfolio returns information
# The ETF SPY will represent the market


In [ ]:
#Calculate the Portfolio Standard Deviation for Tech: use (weightAMZN^2 * MeanAMZN^2 + weightRTH^2 * MeanRTH^2 + 2*weightAMZN*weightRTH*stdevAMZN*stdevRTH*covAMZN,RTH)


In [ ]:
#Calculate the Portfolio Standard Deviation for Real Estate


In [ ]:
#Calculate the Portfolio Standard Deviation for Energy


In [ ]:
# Using the Pandas rolling function in conjunction with the var function, 
# calculate the 30-day rolling variance for the S&P 500 using tech daily returns information

# Visualize the 30-day rolling variance of the S&P 500
# Be sure to adjust the figure size and add a title


In [ ]:
#Monte Carlo Simulation


In [ ]:
#Box and Whisker Plot


In [ ]:
#HVPlot


In [ ]:
#bar plot comparing returns of all sectors 

In [ ]:
#bar plot comparing returns of all portfolio